In [ ]:
import pandas as pd, pickle, numpy as np
pd.options.mode.copy_on_write = True
from config_path import PATH_CLEAN, PATH_API
from functions_shared import chunkify, work_csv
from step7_persons.prep_persons import persons_preparation
from step7_persons.affiliations import persons_affiliation

CSV_DATE='20250121'
# persons_preparation(CSV_DATE)

PATH_PERSONS=f"{PATH_API}persons/"
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
perso_app = pd.read_pickle(f"{PATH_CLEAN}persons_applicants.pkl")

#provisoire
project=pd.read_pickle(f"{PATH_CLEAN}projects_current.pkl") 
perso_part=perso_part.merge(project[['project_id', 'stage', 'destination_code', 'thema_code']], how ='left', on=['project_id', 'stage'])
perso_app=perso_app.merge(project[['project_id', 'stage', 'destination_code', 'thema_code']], how ='left', on=['project_id', 'stage'])


#PREPRATION data for request openalex
# lvar=['contact','orcid_id','country_code','destination_code','thema_code','nationality_country_code']
# pp = pd.concat([perso_part[lvar].drop_duplicates(), perso_app[lvar].drop_duplicates()], ignore_index=True)

# mask=((pp.country_code=='FRA')|(pp.nationality_country_code=='FRA')|(pp.destination_code.isin(['COG', 'PF', 'STG', 'ADG', 'POC','SyG', 'PERA', 'SJI'])))&(~(pp.contact.isnull()&pp.orcid_id.isnull()))
# pp=pp.loc[mask].sort_values(['country_code','orcid_id'], ascending=False).drop_duplicates()
# print(f"size pp: {len(pp)}, info sur pp with orcid: {len(pp.loc[pp.orcid_id.isnull()])}")


# pp=pp[['contact', 'orcid_id']].fillna('').drop_duplicates().sort_values('orcid_id', ascending=False)
# data_chunks=list(chunkify(pp, 2000))
# for i in range(0, len(data_chunks)):
#     print(f"Loop {i}, size data_chunks: {len(data_chunks)}")
#     # print(type(data_chunks))
#     df_temp = data_chunks[i]
#     persons_affiliation(df_temp, i, PATH_PERSONS)

# #Return openalex results
# pers_api=[]
# # for i in range(0, len(data_chunks)):
# for i in range(0, 7):
#     with open(f"{PATH_PERSONS}persons_author_{i}.pkl", 'rb') as f:
#         globals()[f"pers_api{i}"] = pickle.load(f)
#         if globals()[f"pers_api{i}"]==[]:
#             print(f"- empty list: {globals()[f"pers_api{i}"]}")
#         else:
#             pers_api.extend(globals()[f"pers_api{i}"])
# with open(f'{PATH_PERSONS}persons_authors_{CSV_DATE}.pkl', 'wb') as f:
#     pickle.dump(pers_api, f)

with open(f'{PATH_PERSONS}persons_authors_{CSV_DATE}.pkl', 'rb') as f:
    pers_api=pickle.load(f)

pers_api=pd.json_normalize(pers_api)
pers_api=pers_api[~pers_api.astype(str).duplicated()]

# remove name null and author found by orcid but without institutions
pers_api=pers_api[(~pers_api.name.isnull())]
# pers_api=pers_api[~((pers_api.match=='orcid')&(pers_api.affiliations.str.len()==0))]


In [ ]:
pers_api.loc[(pers_api.match=='name')&(~pers_api['ids.orcid'].isnull()), ['orcid','ids.orcid']]

In [ ]:

#traitement au niveau ORCID
# pers_orcid=pers_api.loc[(pers_api.match=='orcid')&(pers_api.affiliations.str.len()!=0)]
# pers_orcid

def persons_results_clean(df, match):
    df=df.loc[(df.match==match)&(df.affiliations.str.len()>0)]
    print(f"-1 size {match} with affiliations: {len(df)}")
    df=df.explode('affiliations')
    df=df.join(pd.json_normalize(df['affiliations'], max_level=1))

    df.columns = df.columns.str.replace(r"[.*_]+", '_', regex=True)
    df = (df
                .rename(columns={'institution_display_name':'institution_name',
                                'institution_country_code':'country_code'})
                .drop(columns=['institution_type','institution_lineage','affiliations',
                                'topics','topics',
                                'ids_openalex','ids_scopus','ids_twitter']))
    print(f"-2 size {match} after explode affiliations: {len(df)}")

    df['display_name_alternatives']=df['display_name_alternatives'].map(lambda x: ';'.join(filter(None, x)))
    df=df[~df.astype(str).duplicated()]

    for i in ['ids_orcid', 'institution_ror']:
        df.loc[~df[i].isnull(), i] = df.loc[~df[i].isnull()][i].str.split("/").str[-1]
    print(f"-3 size {match} cleaned: {len(df)}")
    return df

pers_orcid=persons_results_clean(pers_api, 'orcid')
pers_name=persons_results_clean(pers_api, 'name')


In [ ]:
from step8_referentiels.paysage import paysage_prep
from config_path import PATH
DUMP_PATH=f'{PATH}referentiel/'
paysage = paysage_prep(DUMP_PATH)
paysage

In [ ]:
pers_api[pers_api.name=='zytkiewicz malgorzata']

In [ ]:
def openalex_name(author):
    import time, requests
    try:
        url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
        nb_openalex=requests.get(url).json().get("meta").get('count')
        dl=[]
        if nb_openalex>0:
            print(nb_openalex)
            for n in range(0, nb_openalex): 
                author_openalex = requests.get(url).json().get("results")[n]
                if author_openalex.get('affiliations')!=[]:
                    result=author | {'display_name':author_openalex.get('display_name'), 
                                    'openalex_id':author_openalex.get('id'), 
                                    'affiliations':author_openalex.get('affiliations'),
                                    'topics':author_openalex.get('topics'),
                                    'x_concepts':author_openalex.get('x_concepts'), 
                                    'ids':author_openalex.get('ids'), 
                                    'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                                    'match':'name'}
                    dl.append(result)
        return dl
    
    # except requests.exceptions.HTTPError as http_err:
    #     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")    
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")

def openalex_orcid(author):
    from config_api import openalex_usermail
    import requests, time
    try:
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto={openalex_usermail}"
        author_openalex = requests.get(url).json()
        result = author | {'display_name':author_openalex.get('display_name'), 
                           'openalex_id':author_openalex.get('id'), 
                           'affiliations':author_openalex.get('affiliations'), 
                           'topics':author_openalex.get('topics'),  
                           'x_concepts':author_openalex.get('x_concepts'), 
                           'ids':author_openalex.get('ids'), 
                           'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                           'match':'orcid'}
        return result
    
    # except requests.exceptions.HTTPError as http_err:
    #     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
    #     return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author         
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author
    


    

In [ ]:
import time, pickle, requests
from step7_persons.affiliations import openalex_name, openalex_orcid
from config_path import PATH_CLEAN, PATH_API, PATH_WORK
print(time.strftime("%H:%M:%S"))
rlist=[]

try:


    author = {
    "name": "caye pierre'",
    "orcid": ""
    }

    if author.get("orcid"):
        result = openalex_orcid(author)
        if result.get('match'):
            rlist.append(result)
        else:
            result = openalex_name(author)
            if result:
                rlist.extend(result)
    if author.get("orcid")=='':
        result = openalex_name(author)
        if result:
            rlist.extend(result)

    nf=f"persons_author"
    with open(f'{PATH_WORK}test.pkl', 'wb') as f:
        pickle.dump(rlist, f)
    print(time.strftime("%H:%M:%S"))

except requests.exceptions.HTTPError as http_err:
    print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
    nf=f"persons_author"
    with open(f'{PATH_WORK}test.pkl', 'wb') as f:
        pickle.dump(rlist, f)
    print(time.strftime("%H:%M:%S"))

In [ ]:

with open(f'{PATH_WORK}test.pkl', 'rb') as f:
    test = pickle.load(f)
test

## return results from apenalex

1 - à partir d'orcid
2 - à partir du nom car orcid pas dans openalex -> vérifier que même personne ?
3 - à partir du nom car orcid non renseigné


In [ ]:
def my_country_code():
    import pycountry, pandas as pd, json, numpy as np, warnings
    warnings.simplefilter("ignore")
    pycountry.countries.add_entry(alpha_2="XK", alpha_3="XXK", name="Kosovo")
    pycountry.countries.add_entry(alpha_2="UK", alpha_3="GBR", name="United Kingdom")
    pycountry.countries.add_entry(alpha_2="EL", alpha_3="GRC", name="Greece")
    pycountry.countries.add_entry(alpha_2="AN", alpha_3="ANT", name="Netherlands Antilles (former 2011)")
    dict1 = [c.__dict__['_fields'] for c in list(pycountry.countries)]
    df = (pd.DataFrame(dict1)[['alpha_2', 'alpha_3', 'name']]
                .rename(columns={'alpha_2':'iso2', 'alpha_3':'iso3', 'name':'country_name_en'})
                .drop_duplicates()
                .assign(parent_iso2=np.nan)
        )

    list_var=['iso2']
    ccode=json.load(open("data_files/countries_parent.json"))
    for c in list_var:
        for k,v in ccode.items():
            df.loc[df[c]==k, 'parent_iso2'] = v

    df.loc[df.parent_iso2.isnull(), 'parent_iso2'] = df.loc[df.parent_iso2.isnull(), 'iso2']
    df=(df.merge(df[['iso2','iso3']].drop_duplicates().rename(columns={'iso2':'parent_iso2','iso3':'parent_iso3'}), 
                    how='left', on='parent_iso2'))

    print(len(df))
    return df
my_countries=my_country_code()

In [ ]:
tmp1=pers_api.merge(my_countries[['iso2', 'iso3', 'parent_iso3']].drop_duplicates(),how='left', left_on='country_code', right_on='iso2')
if any(tmp1.country_code.isnull()):
    print(tmp1.loc[tmp1.country_code.isnull(), ['name', 'orcid', 'institution_ror', 'institution_name']])

In [ ]:
pers_api.mask(pers_api=='', inplace=True)
tmp1=pers_api.merge(my_countries, how='left', left_on='country_code', right_on='iso2')
# pers_api=country_clean(pers_api, ['country_code'])
tmp1=pers_api.loc[pers_api.match=='orcid']
len(tmp1)


In [ ]:
print(f"{len(perso_part)} ; {perso_part.columns}")

In [ ]:
tmp_part=perso_part.merge(tmp1, how='inner', left_on=['orcid_id'], right_on=['orcid'], suffixes=('','_api'))
tmp_part.loc[tmp_part.country_code!=tmp_part.country_code_api]

In [ ]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
data

In [ ]:
def openalex_name(author):
    url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
    nb_openalex=requests.get(url).json().get("meta").get('count')
    result={}
    if nb_openalex>0:
        for n in range(nb_openalex): 
            author_openalex = requests.get(url).json().get("results")[n]
            result.update({'display_name':author_openalex.get('display_name'), 'openalex_id':author_openalex.get('id'), 'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'ids':author_openalex.get('ids'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
            author.update(result)
            df=pd.concat([df, pd.json_normalize(author)])
    return df

In [ ]:
from config_path import PATH_API
import os, pandas as pd
 

for racine, repertoires, fichiers in os.walk(PATH_API):
    print(f"{racine}, {repertoires}, {fichiers}")
    for fichier in fichiers:
        if fichier.startswith('persons'):
            print(os.path.join(racine, fichier))
            globals()[f"{fichier}"]= pd.read_pickle(os.path.join(racine, fichier))


In [ ]:
fl=['persons_author.pkl', 'persons_author_10000.pkl', 'persons_author_12000.pkl', 'persons_author_2000.pkl', 'persons_author_4000.pkl', 'persons_author_6000.pkl', 'persons_author_8000.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']
for racine, repertoires, fichiers in os.walk(PATH_API):
    for i in fl:
        name=f"{i}".split('.')[0]
        print(name)
        globals()[name] = pd.read_pickle(os.path.join(racine, fichier))

In [ ]:
pers=pd.concat([persons_author_10000, persons_author_12000, persons_author_2000, persons_author_4000, persons_author_6000, persons_author_8000], ignore_index=True)

fl=['persons_author.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']

In [ ]:
def openalex_orcid(author):
    # from config_api import openalex_usermail
    import requests, time
    try:
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto=zmenesr@gmail.com"
        author_openalex = requests.get(url).json()
        result = author | {'display_name':author_openalex.get('display_name'), 
                           'openalex_id':author_openalex.get('id'), 
                           'affiliations':author_openalex.get('affiliations'), 
                           'topics':author_openalex.get('topics'),  
                           'x_concepts':author_openalex.get('x_concepts'), 
                           'ids':author_openalex.get('ids'), 
                           'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                           'match':'orcid'}
        return result
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author

In [ ]:
def openalex_name(author):
    import time
    try:
        url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
        nb_openalex=requests.get(url).json().get("meta").get('count')
        dl=[]
        if nb_openalex>0:
            print(nb_openalex)
            for n in range(nb_openalex): 
                author_openalex = requests.get(url).json().get("results")[n]
                if author_openalex.get('affiliations')!=[]:
                    result=author | {'display_name':author_openalex.get('display_name'), 
                                    'openalex_id':author_openalex.get('id'), 
                                    'affiliations':author_openalex.get('affiliations'),
                                    'topics':author_openalex.get('topics'),
                                    'x_concepts':author_openalex.get('x_concepts'), 
                                    'ids':author_openalex.get('ids'), 
                                    'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                                    'match':'name'}
                    dl.append(result)
        return dl
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author
    

In [ ]:
from config_path import PATH_CLEAN
# from functions_shared import chunkify
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
# pp = pd.concat([perso_part[['contact', 'orcid_id', 'country_code']].drop_duplicates(), perso_app[['contact', 'orcid_id', 'country_code']].drop_duplicates()], ignore_index=True)
pp = perso_part[['contact', 'orcid_id', 'country_code']].fillna('')
pp=pp.loc[(pp.country_code=='FRA')].sort_values('orcid_id', ascending=False)
pp
data_chunks=list(chunkify(pp, 10000))
for i in range(0, len(data_chunks)):
    print(f"Loop {i}, size data_chunks: {len(data_chunks)}")
    # print(type(data_chunks))
    df_temp = data_chunks[i]
    

In [ ]:
import pickle, pandas as pd
from config_path import PATH_API
pers_api=[]
for i in range(1,3):
    print(i)
    with open(f"{PATH_API}fr_persons_author_{i}.pkl", 'rb') as f:
        globals()[f"pers_api{i}"] = pickle.load(f)
    pers_api.extend(globals()[f"pers_api{i}"])


In [ ]:
pers_api=pd.json_normalize(pers_api1, record_path=['affiliations'], meta=['name', 'orcid', 'display_name', 'openalex_id',  'match',  ["ids", "orcid"]],
        errors='ignore')

pers_api.columns = pers_api.columns.str.replace(r"[.*_]+", '_', regex=True)

pers_api = (pers_api
            .rename(columns={
                    'institution_country_code':'country_code'})
            .drop(columns=['institution_type','institution_lineage']))

for i in ['ids_orcid', 'institution_ror']:
    pers_api.loc[~pers_api[i].isnull(), i] = pers_api.loc[~pers_api[i].isnull()][i].str.split("/").str[-1]

In [ ]:
from config_path import PATH_CLEAN
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
perso_app = pd.read_pickle(f"{PATH_CLEAN}persons_applicants.pkl")

In [ ]:
with open(f"{PATH_API}persons_author.pkl", 'rb') as f:
    author_orcid = pickle.load(f)

In [ ]:
author_orcid=pd.json_normalize(author_orcid, record_path=['affiliations'], meta=['name','orcid', 'display_name', 'ids', 'match'])
author_orcid